In [1]:
import pandas as pd
import numpy as np

In [2]:
teacher_df = pd.read_csv('data/CY08MSP_TCH_KOR_VARS select.csv')

In [3]:
teacher_df = teacher_df.iloc[:, :-1]  # 마지막 열 제거
teacher_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614 entries, 0 to 3613
Columns: 526 entries, CNT to VER_DAT
dtypes: float64(117), int64(407), object(2)
memory usage: 14.5+ MB


In [4]:
teacher_vars = pd.read_csv('data/TCH_VARS_original.csv')

In [5]:
teacher_vars.head()

,변수,설명,측도,결측값
0,CNT,Country code 3-character,명목(N),없음
1,CNTTCHID,Intl. Teacher ID,명목(N),없음
2,TCHTYPE,Teacher Type,명목(N),"7, 8, 9"
3,TC001Q01NA,Are you female or male?,명목(N),95 - 99
4,TC002Q01NA,How old are you?,척도(S),95 - 99


In [6]:
(np.array(teacher_df.columns) == np.array(teacher_vars['변수'])).all()

np.True_

In [7]:
teacher_vars_selected = pd.read_csv('data/TCH_VARS_selected.csv')
teacher_vars_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   변수      117 non-null    object
dtypes: object(1)
memory usage: 1.0+ KB


In [8]:
teacher_columns_selected = teacher_vars_selected['변수'].to_list()
teacher_columns_selected[:5]

['TCHTYPE', 'TC001Q01NA', 'TC002Q01NA', 'TC007Q02NA', 'TC215Q01JA']

In [9]:
teacher_df_selected = teacher_df[teacher_columns_selected]
teacher_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614 entries, 0 to 3613
Columns: 117 entries, TCHTYPE to NEGSYMPT
dtypes: float64(33), int64(84)
memory usage: 3.2 MB


In [10]:
teacher_df_selected.head()

,TCHTYPE,TC001Q01NA,TC002Q01NA,TC007Q02NA,TC215Q01JA,TC216Q01JA,TC216Q02JA,TC216Q03JA,TC216Q04JA,TC216Q05JA,...,CREATPED,CAPCON,AFFECT,TRUST,OVERLOAD,AUTONOMY,LEADSHIP,OCSTRESS,STRESS,NEGSYMPT
0,2,1,56,29,56,17,18,999,10,1,...,-0.8781,-1.8075,-1.6162,-1.3334,0.9153,-1.3328,-2.0301,0.2852,0.3418,1.0584
1,1,1,28,5,22,17,1,1,0,1,...,-0.6469,1.1064,0.7089,1.1913,-0.6376,-0.1241,0.6794,-0.3719,-0.1074,0.1544
2,1,2,34,6,27,19,1,1,2,1,...,-1.4753,-1.1389,-1.0058,-0.8878,-0.1250,-1.7615,-0.3898,0.2094,0.4058,-0.3043
3,2,2,42,7,60,10,5,2,2,30,...,-0.4514,-0.1377,-0.0544,-0.4751,-0.2848,-0.7672,0.6432,-0.3301,-0.5373,-0.7681
4,1,2,31,3,44,17,4,1,2,5,...,-0.3913,-0.1377,-0.0544,-0.4751,0.4214,1.3140,0.2576,0.1592,0.8580,-0.1894


- 95, 99, 9999 등은 결측치
- 열마다 결측치 값이 다름

In [11]:
teacher_df_selected.describe(percentiles=[0.9, 0.92, 0.95, 0.99])

,TCHTYPE,TC001Q01NA,TC002Q01NA,TC007Q02NA,TC215Q01JA,TC216Q01JA,TC216Q02JA,TC216Q03JA,TC216Q04JA,TC216Q05JA,...,CREATPED,CAPCON,AFFECT,TRUST,OVERLOAD,AUTONOMY,LEADSHIP,OCSTRESS,STRESS,NEGSYMPT
count,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,...,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000
mean,1.684283,1.573879,43.959878,16.259546,350.731599,28.769231,27.575816,48.882125,58.855285,52.203929,...,0.825549,0.967207,0.781324,0.846966,1.453154,1.155628,1.357440,1.490599,1.458462,1.674074
std,0.464865,3.660585,9.821958,11.012970,5508.108954,110.347336,135.708507,207.872831,228.213614,213.661046,...,11.185266,11.292714,10.945370,10.934424,11.343863,10.534830,12.175294,11.094712,11.934829,12.136062
min,1.000000,1.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-3.804800,-3.894700,-3.139800,-3.009000,-3.506700,-3.401700,-2.030100,-3.761100,-2.721200,-2.325700
50%,2.000000,1.000000,44.000000,16.000000,40.000000,16.000000,8.000000,3.000000,3.000000,3.000000,...,-0.619800,-0.137700,-0.054400,-0.475100,0.167600,-0.161800,-0.294600,0.209400,0.167100,0.251400
90%,2.000000,2.000000,57.000000,31.000000,60.000000,20.000000,18.000000,10.000000,10.000000,10.000000,...,0.917300,1.064800,0.853210,1.191300,1.049510,1.344200,0.858900,0.988000,0.963380,1.193170
92%,2.000000,2.000000,58.000000,32.000000,60.000000,20.000000,20.000000,10.000000,10.000000,13.000000,...,1.056900,1.210300,1.165900,1.557900,1.155900,1.726400,1.130800,1.142872,1.071476,1.319288
95%,2.000000,2.000000,60.000000,34.000000,65.000000,23.000000,23.000000,20.000000,999.000000,54.150000,...,1.766800,2.312300,1.740600,1.557900,1.474300,2.565200,1.821400,1.385700,1.277210,1.670440
99%,2.000000,2.000000,62.000000,37.870000,193.500000,999.000000,999.000000,999.000000,999.000000,999.000000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
max,2.000000,99.000000,99.000000,99.000000,99999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [12]:
teacher_vars_selected

,변수
0,TCHTYPE
1,TC001Q01NA
2,TC002Q01NA
3,TC007Q02NA
4,TC215Q01JA
...,...
112,AUTONOMY
113,LEADSHIP
114,OCSTRESS
115,STRESS


열별로 dtype 을 확인하기 위해 측도와 결측값 종류 추가

In [13]:
teacher_vars_selected_dtypes = teacher_vars_selected.merge(teacher_vars, on='변수', how='left')
teacher_vars_selected_dtypes

,변수,설명,측도,결측값
0,TCHTYPE,Teacher Type,명목(N),"7, 8, 9"
1,TC001Q01NA,Are you female or male?,명목(N),95 - 99
2,TC002Q01NA,How old are you?,척도(S),95 - 99
3,TC007Q02NA,How many years of work experience do you have:...,척도(S),95 - 99
4,TC215Q01JA,"During most recent complete calendar week, app...",명목(N),99995 - 99999
...,...,...,...,...
112,AUTONOMY,Teachers' work autonomy (WLE),척도(S),95.0000 - 99.0000
113,LEADSHIP,School leadership (WLE),척도(S),95.0000 - 99.0000
114,OCSTRESS,Occupational stress (WLE),척도(S),95.0000 - 99.0000
115,STRESS,Sources of stress (WLE),척도(S),95.0000 - 99.0000


먼저 결측치를 결측치로 만들어야 함.

In [14]:
teacher_vars_selected_dtypes['결측값'].value_counts()

결측값
5, 6, 7, 8, 9        41
95.0000 - 99.0000    33
95 - 99              24
995 - 999            15
99995 - 99999         2
7, 8, 9               1
없음                    1
Name: count, dtype: int64

In [25]:
teacher_vars_selected_dtypes

,변수,설명,측도,결측값
0,TCHTYPE,Teacher Type,명목(N),"7, 8, 9"
1,TC001Q01NA,Are you female or male?,명목(N),95 - 99
2,TC002Q01NA,How old are you?,척도(S),95 - 99
3,TC007Q02NA,How many years of work experience do you have:...,척도(S),95 - 99
4,TC215Q01JA,"During most recent complete calendar week, app...",명목(N),99995 - 99999
...,...,...,...,...
112,AUTONOMY,Teachers' work autonomy (WLE),척도(S),95.0000 - 99.0000
113,LEADSHIP,School leadership (WLE),척도(S),95.0000 - 99.0000
114,OCSTRESS,Occupational stress (WLE),척도(S),95.0000 - 99.0000
115,STRESS,Sources of stress (WLE),척도(S),95.0000 - 99.0000


In [15]:
import re

In [23]:
delimiter = r' - |, '
re.split(delimiter, '7, 8, 9')

['7', '8', '9']

In [33]:
def parse_missing_codes(code_str):
    """'99, 999' -> [99.0, 999.0] 변환"""
    if code_str == '없음':
        return [np.nan]
    # elif pd.isna(code_str) or code_str == '':
        # return []
    else:
        missing_range = [float(x.strip()) for x in re.split(delimiter, str(code_str))]
        return np.arange(missing_range[0], missing_range[-1] + 1)




# 변수명-결측코드 매핑 생성
missing_map = dict(zip(
    teacher_vars_selected_dtypes['변수'],
    teacher_vars_selected_dtypes['결측값'].apply(parse_missing_codes)  # '결측코드' 컬럼이 있다면
))

In [34]:
missing_map

{'TCHTYPE': array([7., 8., 9.]),
 'TC001Q01NA': array([95., 96., 97., 98., 99.]),
 'TC002Q01NA': array([95., 96., 97., 98., 99.]),
 'TC007Q02NA': array([95., 96., 97., 98., 99.]),
 'TC215Q01JA': array([99995., 99996., 99997., 99998., 99999.]),
 'TC216Q01JA': array([995., 996., 997., 998., 999.]),
 'TC216Q02JA': array([995., 996., 997., 998., 999.]),
 'TC216Q03JA': array([995., 996., 997., 998., 999.]),
 'TC216Q04JA': array([995., 996., 997., 998., 999.]),
 'TC216Q05JA': array([995., 996., 997., 998., 999.]),
 'TC216Q06JA': array([995., 996., 997., 998., 999.]),
 'TC216Q07JA': array([995., 996., 997., 998., 999.]),
 'TC216Q08JA': array([995., 996., 997., 998., 999.]),
 'TC218Q01JA': array([95., 96., 97., 98., 99.]),
 'TC150Q01HA': array([95., 96., 97., 98., 99.]),
 'TC150Q02HA': array([95., 96., 97., 98., 99.]),
 'TC150Q03HA': array([95., 96., 97., 98., 99.]),
 'TC150Q04HA': array([95., 96., 97., 98., 99.]),
 'TC150Q06HA': array([95., 96., 97., 98., 99.]),
 'TC150Q08HA': array([95., 96.

In [ ]:
teacher_vars_selected_dtypes['측도'].value_counts()

In [ ]:
mask_order = teacher_vars_selected_dtypes['측도'] == '순서(O)'
mask_nominal = teacher_vars_selected_dtypes['측도'] == '명목(N)'
mask_scale = teacher_vars_selected_dtypes['측도'] == '척도(S)'

In [ ]:
teacher_vars_selected_dtypes['측도'].value_counts()

In [ ]:
cols = teacher_df_selected.columns[(~mask_scale).to_numpy()]
teacher_df_selected.loc[:, cols] = teacher_df_selected.loc[:, cols].astype(str)

In [ ]:
teacher_df_selected.info()

In [ ]:
teacher_df_selected.select_dtypes(include='int64')

In [ ]:
teacher_df_selected.select_dtypes(include='float64')

In [ ]:
teacher_df_selected.describe()

In [ ]:
teacher_df_selected['TC258Q01JA'].value_counts()

In [ ]:
teacher_df_selected['PRPDT'].value_counts()

In [ ]:
teacher_df_selected.loc[:, 'ADAPTINSTR']

In [ ]:
targets = ['ADAPTINSTR', 'FEEDBINSTR']

In [ ]:
teacher_df_selected.loc[:, targets].describe()

In [ ]:
teacher_df_selected.loc[:, 'ADAPTINSTR'].value_counts()

In [ ]:
teacher_df_selected.loc[:, 'FEEDBINSTR'].value_counts()

In [ ]:
# 모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# 전처리
from sklearn.preprocessing import OneHotEncoder

# 컬럼 변환: 파이프라인 활용
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer

from sklearn.pipeline import Pipeline

# 모델 평가
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


In [ ]:
teacher_df_selected.isna().sum().sum()

## RandomForest 학습

In [ ]:
X_2022 = teacher_df_selected.drop(targets, axis=1).copy()
y_2022 = teacher_df_selected.loc[:, targets].copy()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_2022, y_2022, test_size=0.2, random_state=42)